# Analysis of Video and Image Data for cropping and texture features
Author(s): Miguel Xochicale @mxochicale    
Contributor(s): 

## History
* 17th May 2022: Add prototype

## Summary


### How to run the notebook
1. Go to repository path: `$HOME/repositories/`
Open repo in pycharm and in the terminal type:
```
git checkout main # or the branch
git pull # to bring a local branch up-to-date with its remote version
```

2. Launch Notebook server. Go to you repository path: cd $HOME/repositories/ and type in the pycharm terminal:
```
mamba activate *VE 
jupyter notebook --browser=firefox
```
which will open your web-browser.

## References 
1. https://stackoverflow.com/questions/45704999/how-to-convert-vector-wrapped-as-string-to-numpy-array-in-pandas-dataframe
2. https://github.com/YuxinZhaozyx/pytorch-VideoDataset/blob/master/datasets.py (Future work)
3. https://stackoverflow.com/questions/65446464/how-to-convert-a-video-in-numpy-array
4. https://matplotlib.org/stable/gallery/specialty_plots/mri_with_eeg.html#sphx-glr-gallery-specialty-plots-mri-with-eeg-py 
5. https://www.researchgate.net/publication/326881329_Medical_image_security_enhancement_using_two_dimensional_chaotic_mapping_optimized_by_self-adaptive_grey_wolf_algorithm 

    

## Setting imports and datasets paths

In [ ]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import skimage
from skimage.feature import graycomatrix, graycoprops
from typing import Tuple, List

HOME_PATH = os.path.expanduser(f'~')
USERNAME = os.path.split(HOME_PATH)[1]
REPOSITORY_PATH='repositories/in2research2023'


###########################
###SET DATA_PATH 
DATA_PATH='scripts/sensor-fusion'
FULL_REPO_DATA_PATH = HOME_PATH +'/' + REPOSITORY_PATH +'/' + DATA_PATH


###########################
### experiments_13-Jul-2023
AVI_FILE_01 = 'test01.avi'
CSV_FILE_01 = 'test01.avi.csv'

AVI_FILE_02 = 'test02.avi'
CSV_FILE_02 = 'test02.avi.csv'


FULL_PATH_AND_AVI_FILE_01 = os.path.join(FULL_REPO_DATA_PATH , AVI_FILE_01)
FULL_PATH_AND_CSV_FILE_01 = os.path.join(FULL_REPO_DATA_PATH , CSV_FILE_01)

FULL_PATH_AND_AVI_FILE_02 = os.path.join(FULL_REPO_DATA_PATH , AVI_FILE_02)
FULL_PATH_AND_CSV_FILE_02 = os.path.join(FULL_REPO_DATA_PATH , CSV_FILE_02)


## Printing Versions and paths
print(f'PyTorch Version: {torch.__version__}')
print(f'pandas Version: {pd.__version__}')
print(f'seaborn Version: {sns.__version__}')
print(f'numpy Version: {np.__version__}')
print(f'cv2 Version: {cv2.__version__}')
print(f'skimage Version: {skimage.__version__}')

print(f'FULL_REPO_DATA_PATH: {FULL_REPO_DATA_PATH}')
print(f'FULL_PATH_AND_CSV_FILE: {FULL_PATH_AND_CSV_FILE_01}')
print(f'FULL_PATH_AND_AVI_FILE: {FULL_PATH_AND_AVI_FILE_01}')

print(f'FULL_PATH_AND_CSV_FILE: {FULL_PATH_AND_CSV_FILE_02}')
print(f'FULL_PATH_AND_AVI_FILE: {FULL_PATH_AND_AVI_FILE_02}')




In [ ]:
def msec_to_timestamp(current_timestamp: float) -> Tuple[float]:
    """
    Convert millisecond variable to a timestamp variable with the format minutes, seconds and milliseconds
    """
    minutes = int(current_timestamp / 1000 / 60)
    seconds = int(np.floor(current_timestamp / 1000) % 60)
    ms = current_timestamp - np.floor(current_timestamp / 1000) * 1000

    return minutes, seconds, '{:.3f}'.format(ms), '{:02d}:{:02d}:{:.3f}'.format(minutes, seconds, ms)

    
def masks_us_image(image_frame_array_1ch: np.ndarray) -> np.ndarray:
    """
    Hard mask pixels outside of scanning sector
    """
    mask = np.zeros_like(image_frame_array_1ch)

    #                 top-left, 
    #                     top-right, 
    #                           bottom-right
    #                                             arc 
    #                                                                         bottom-left
    
    #x_data = np.array([250, 380, 572,           454,321,165                   ,60])
    #y_data = np.array([30, 30, 320,             389,421,382                   ,320 ])
    
    
    x_data = np.array([250, 380, 572,           
                       597,580,561,537,515,491,474,436,395,366,333,293,249,207,166,     97,77,61,42  #arc
                       ,60])#bottom-left
    y_data = np.array([30, 30, 320,             
                       368,380,392,404,416,424,430,440,448,451,452,453,446,439,426,     398,388,380,366  #arc
                       ,320 ]) #bottom-left
    scan_arc_mask_v01 = np.vstack((x_data, y_data)).astype(np.int32).T


    
    #caliper_scale_mask = np.array([(1770, 120), (1810, 120), (1810, 930), (1770, 930)])
    cv2.fillPoly(mask, [scan_arc_mask_v01],
                (255, 255, 0))
    maskedImage = cv2.bitwise_and(image_frame_array_1ch, image_frame_array_1ch, mask=mask)

    return maskedImage



def GLCMs(img):
    """
    Calculating gray level co-occurrence matrices (GLCMs)
    https://scikit-image.org/docs/dev/auto_examples/features_detection/
           plot_glcm.html#sphx-glr-auto-examples-features-detection-plot-glcm-py

    Haralick, RM.; Shanmugam, K., “Textural features for image classification” 
        IEEE Transactions on systems, man, and cybernetics 6 (1973): 610-621. 
        [DOI:10.1109/TSMC.1973.4309314](https://doi.org/10.1109/TSMC.1973.4309314)
    PDF: http://haralick-org.torahcode.us/journals/TexturalFeaturesHaralickShanmugamDinstein.pdf
    google-citations: https://scholar.google.com/scholar?cites=
                        13863271628667072083&as_sdt=2005&sciodt=0,5&hl=en
   
    """
    return graycomatrix(img,[1],[0],levels=256)

def Contrast(R):
    ct = graycoprops(R,prop='contrast')
    return ct

def Correlation(R):
    cn = graycoprops(R,prop='correlation')
    return cn

def Dissimilarity(R):
    d = graycoprops(R,prop='dissimilarity')
    return d

def Energy(R):
    e = graycoprops(R,prop='energy')
    return e

def Homogeneity(R):
    h = graycoprops(R,prop='homogeneity')
    return h

def ASM(R):
    a = graycoprops(R,prop='ASM')
    return a



def video_to_tensor(FULL_PATH_AND_AVI_FILE, start_frame_number, end_frame_number):
    cap = cv2.VideoCapture(FULL_PATH_AND_AVI_FILE)
    if cap.isOpened() == False:
        print('[ERROR] [ViewVideoDataset.__getitem__()] Unable to open video ' + FULL_PATH_AND_AVI_FILE)
        exit(-1)

    # Get parameters of input video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(np.ceil(cap.get(cv2.CAP_PROP_FPS)))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Print video features
    print(f'  ')
    print(f'  ')
    print(f'  VIDEO_FEATURES')
    print(f'    video_name={FULL_PATH_AND_AVI_FILE}')
    print(f'    Frame_height={frame_height}, frame_width={frame_width} fps={fps} nframes={frame_count} ')
    print(f'  ')
    print(f'  ')
    # # #         if start_frame_number >= end_frame_number:
    # # #             raise Exception("start frame number must be less than end frame number")

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)
    frames_numpy_ndarray=[]
    frames_torch = []
    frames_timestamp = []

    # pbar = tqdm(total=total_number_of_frames - 1)
    while cap.isOpened():
        success, image_frame_3ch_i = cap.read()

        if not success and len(frames_torch) < 1:
            print(f'[ERROR] {FULL_PATH_AND_AVI_FILE}')
            exit(-1)
            break

        if cap.get(cv2.CAP_PROP_POS_FRAMES) >= end_frame_number:
            break

        frame_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
        current_frame_timestamp = msec_to_timestamp(frame_msec)
        #print(current_frame_timestamp)#(0, 33, '333.333', '00:33:333.333')
        #print(current_frame_timestamp[2])#'00:33:333.333'
        #print(current_frame_timestamp[3])#00:33:333.333


        #(H x W x C) to (C x H x W)
        #print(type(image_frame_3ch_i))#    <class 'numpy.ndarray'>
        #print(image_frame_3ch_i.shape)#(480, 640, 3)
        image_frame_1ch_i = cv2.cvtColor(image_frame_3ch_i, cv2.COLOR_BGR2GRAY)

        frames_numpy_ndarray.append(image_frame_1ch_i)
        #frames_numpy_ndarray.append(masks_us_image(image_frame_1ch_i))

        frames_timestamp.append(current_frame_timestamp[3])

    #     frame_torch = torch.from_numpy(image_frame_3ch_i).float()
    #     frame_torch = frame_torch.squeeze()  # Fake batch dimension to be "H,W,C"
    #     print(type(frame_torch))#<class 'torch.Tensor'>
    #     print(frame_torch.shape)#torch.Size([480, 640, 3])

    # # #             cropped_image_frame_ = cropped_frame(masked_frame, self.crop_bounds)
    # # #             frame_torch = ToImageTensor(cropped_image_frame_)

    video = np.stack(frames_numpy_ndarray, axis=0) # dimensions (Fi, H, W, C)
    cap.release()

    return video, frames_timestamp

# Reading video and plotting frames

In [ ]:

start_frame_number = 000
end_frame_number = 500
total_number_of_frames = end_frame_number - start_frame_number

video_01, frames_timestam_01 = video_to_tensor(FULL_PATH_AND_AVI_FILE_01, start_frame_number, end_frame_number)
print(video_01.shape) 

video_02, frames_timestam_02 = video_to_tensor(FULL_PATH_AND_AVI_FILE_02, start_frame_number, end_frame_number)
print(video_02.shape) 


# Plotting frames

In [ ]:
def display_us_hist(im):
    im=video[frame_i]
    fig = plt.figure(dpi=50, figsize=(15, 5))

    
    ax0 = fig.add_subplot(1, 2, 1)
    ax0.imshow(im, cmap='gray')

    
    ax1 = fig.add_subplot(1, 2, 2)

    #im = np.ravel(im)
    #im = im[np.nonzero(im)]  # Ignore the background
    #im = im / (2**16 - 1)  # Normalize
    #ax1.hist(im, bins=10)
    #https://matplotlib.org/stable/gallery/specialty_plots/
           #mri_with_eeg.html#sphx-glr-gallery-specialty-plots-mri-with-eeg-py
    
    im = np.ravel(im)/256 # Normalize
    im = im[np.nonzero(im)]  # Ignore the background
    ax1.hist(im, bins=256, range=(0.0, 1.0))
    #https://matplotlib.org/stable/tutorials/introductory/images.html#sphx-glr-tutorials-introductory-images-py
    
    ax1.minorticks_on()
    ax1.set_xlabel('Normalised Intensity Values')
    ax1.set_ylabel('Normalised Density')

    plt.tight_layout()
    plt.show()

    
texture_analysis_array_01 = []    
for frame_i in range(len(video_01)):
    
    # Compute Grey-Level-Co-occurrence Matrix    
    R=GLCMs(video_01[frame_i])

    # Calculating texture property Contrast
    con = Contrast(R)

    # Calculating texture property Correlation
    cor = Correlation(R)

    # Calculating texture property Dissimilarity
    dis = Dissimilarity(R)

    # Calculating texture property Energy
    en = Energy(R)

    # Calculating texture property Homogeneity
    homo = Homogeneity(R)
    
    # Calculating texture property ASM
    asm = ASM(R)
    
    texture_analysis_array_01.append([con,cor,dis,en,homo,asm])
    #print(con,cor,dis,en,homo, asm) 
    
    if frame_i % 250 == 0:
        print(f'frame_i: {frame_i}, timestamp {frames_timestam_01[frame_i]}')
        display_us_hist(frame_i)

    
    #plt.savefig('filename'+str(frame_i)+'.png', dpi=300)

# print(texture_analysis_array)  



texture_analysis_array_02 = []    
for frame_i in range(len(video_02)):
    
    # Compute Grey-Level-Co-occurrence Matrix    
    R=GLCMs(video_02[frame_i])

    # Calculating texture property Contrast
    con = Contrast(R)

    # Calculating texture property Correlation
    cor = Correlation(R)

    # Calculating texture property Dissimilarity
    dis = Dissimilarity(R)

    # Calculating texture property Energy
    en = Energy(R)

    # Calculating texture property Homogeneity
    homo = Homogeneity(R)
    
    # Calculating texture property ASM
    asm = ASM(R)
    
    texture_analysis_array_02.append([con,cor,dis,en,homo,asm])
    #print(con,cor,dis,en,homo, asm) 
    
    if frame_i % 250 == 0:
        print(f'frame_i: {frame_i}, timestamp {frames_timestam_02[frame_i]}')
        display_us_hist(frame_i)

    
    #plt.savefig('filename'+str(frame_i)+'.png', dpi=300)

# print(texture_analysis_array)  



# Plotting texture analysis of all frames in the video

In [ ]:
### Video01    
texture_analysis_np_array_01 = np.stack(texture_analysis_array_01, axis=0) 
texture_analysis_np_array_01 = texture_analysis_np_array_01.transpose()
texture_analysis_np_array_01 = texture_analysis_np_array_01.squeeze()
# print(texture_analysis_np_array)
# print(texture_analysis_np_array[0])

df_texture_analysis_01 = pd.DataFrame( 
    {
        'frame_i': np.arange(0, total_number_of_frames-1),
        'Contrast': texture_analysis_np_array_01[0],
        'Correlation': texture_analysis_np_array_01[1],
        'Dissimilarity': texture_analysis_np_array_01[2],
        'Energy': texture_analysis_np_array_01[3],
        'Homogeneity': texture_analysis_np_array_01[4],
        'ASM': texture_analysis_np_array_01[5]

    }
    )
print(df_texture_analysis_01)


# ax = plt.gca()
# ## Comment/uncomment any of the following lines to show however axis you would like to see in the plot
# df_texture_analysis.plot( x = 'frame_i' , y = 'Contrast', ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Correlation' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Dissimilarity' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Energy' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Homogeneity' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'ASM' , ax = ax )
# plt.grid()
# plt.show()

ax = plt.gca()
## Comment/uncomment any of the following lines to show however axis you would like to see in the plot
df_texture_analysis_01.plot( x = 'frame_i' , y = 'Contrast', ax = ax )
plt.grid()
plt.show()

ax = plt.gca()
df_texture_analysis_01.plot( x = 'frame_i' , y = 'Dissimilarity' , ax = ax )
plt.grid()
plt.show()

ax = plt.gca()
df_texture_analysis_01.plot( x = 'frame_i' , y = 'Correlation' , ax = ax )
plt.grid()
plt.show()

ax = plt.gca()
df_texture_analysis_01.plot( x = 'frame_i' , y = 'Energy' , ax = ax )
df_texture_analysis_01.plot( x = 'frame_i' , y = 'Homogeneity' , ax = ax )
df_texture_analysis_01.plot( x = 'frame_i' , y = 'ASM' , ax = ax )
plt.grid()
plt.show()


# #TODO: https://seaborn.pydata.org/generated/seaborn.lineplot.html    


In [ ]:
### Video02    
texture_analysis_np_array_02 = np.stack(texture_analysis_array_02, axis=0) 
texture_analysis_np_array_02 = texture_analysis_np_array_02.transpose()
texture_analysis_np_array_02 = texture_analysis_np_array_02.squeeze()
# print(texture_analysis_np_array)
# print(texture_analysis_np_array[0])

df_texture_analysis_02 = pd.DataFrame( 
    {
        'frame_i': np.arange(0, total_number_of_frames-1),
        'Contrast': texture_analysis_np_array_02[0],
        'Correlation': texture_analysis_np_array_02[1],
        'Dissimilarity': texture_analysis_np_array_02[2],
        'Energy': texture_analysis_np_array_02[3],
        'Homogeneity': texture_analysis_np_array_02[4],
        'ASM': texture_analysis_np_array_02[5]

    }
    )
print(df_texture_analysis_02)


# ax = plt.gca()
# ## Comment/uncomment any of the following lines to show however axis you would like to see in the plot
# df_texture_analysis.plot( x = 'frame_i' , y = 'Contrast', ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Correlation' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Dissimilarity' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Energy' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'Homogeneity' , ax = ax )
# df_texture_analysis.plot( x = 'frame_i' , y = 'ASM' , ax = ax )
# plt.grid()
# plt.show()

ax = plt.gca()
## Comment/uncomment any of the following lines to show however axis you would like to see in the plot
df_texture_analysis_02.plot( x = 'frame_i' , y = 'Contrast', ax = ax )
plt.grid()
plt.show()

ax = plt.gca()
df_texture_analysis_02.plot( x = 'frame_i' , y = 'Dissimilarity' , ax = ax )
plt.grid()
plt.show()

ax = plt.gca()
df_texture_analysis_02.plot( x = 'frame_i' , y = 'Correlation' , ax = ax )
plt.grid()
plt.show()

ax = plt.gca()
df_texture_analysis_02.plot( x = 'frame_i' , y = 'Energy' , ax = ax )
df_texture_analysis_02.plot( x = 'frame_i' , y = 'Homogeneity' , ax = ax )
df_texture_analysis_02.plot( x = 'frame_i' , y = 'ASM' , ax = ax )
plt.grid()
plt.show()


# #TODO: https://seaborn.pydata.org/generated/seaborn.lineplot.html    


## Reading and ploting csv files

In [ ]:
df=pd.read_csv(FULL_PATH_AND_CSV_FILE_01)
# print(df)

# # df = df.rename(columns={'Euler_computed [Roll, Pitch, Yaw]': 'Euler_computed'})
df = df.rename(columns={'Sample number': 'Sample_number'})
df = df.rename(columns={'Euler [Roll, Pitch, Yaw] LPMSB2': 'Euler_LPMSB2'})
df = df.rename(columns={'Quaternions [q0, q1, q2, q3] LPMS-B2': 'Quaternions_LPMSB2'})

df[['A','B','C']]=df.Euler_LPMSB2.str.split(',',expand=True)
df['A']=df['A'].apply(   lambda x: x.replace('[','')   )
df['C']=df['C'].apply(   lambda x:  x.replace(']','')   )
df['A'] = pd.to_numeric(df['A'], errors='coerce')
df['B'] = pd.to_numeric(df['B'], errors='coerce')
df['C'] = pd.to_numeric(df['C'], errors='coerce')

df[['q0','q1','q2', 'q3']]=df.Quaternions_LPMSB2.str.split(',',expand=True)
df['q0']=df['q0'].apply(   lambda x: x.replace('[','')   )
df['q3']=df['q3'].apply(   lambda x:  x.replace(']','')   )
df['q0'] = pd.to_numeric(df['q0'], errors='coerce')
df['q1'] = pd.to_numeric(df['q1'], errors='coerce')
df['q2'] = pd.to_numeric(df['q2'], errors='coerce')
df['q3'] = pd.to_numeric(df['q3'], errors='coerce')

print(df.head())# #Print head of csv


## EULER ANGLES
ndf_a=pd.DataFrame(data=df['Sample_number'])
ndf_a.insert(1, 'Euler_angle', str('alpha'), True)
ndf_a.insert(2, 'Euler_val', df['A'], True)

ndf_b=pd.DataFrame(data=df['Sample_number'])
ndf_b.insert(1, 'Euler_angle', str('beta'), True)
ndf_b.insert(2, 'Euler_val', df['B'], True)

ndf_c=pd.DataFrame(data=df['Sample_number'])
ndf_c.insert(1, 'Euler_angle', str('gamma'), True)
ndf_c.insert(2, 'Euler_val', df['C'], True)

ndf = pd.concat([ndf_a, ndf_b, ndf_c], ignore_index=True)
print(ndf)

sns.lineplot(data=ndf,x='Sample_number', y='Euler_val',hue='Euler_angle',lw=2)
plt.show()


## QUATERNIONS
nqdf_q0=pd.DataFrame(data=df['Sample_number'])
nqdf_q0.insert(1, 'Quaternion', str('q0'), True)
nqdf_q0.insert(2, 'Q_val', df['q0'], True)

nqdf_q1=pd.DataFrame(data=df['Sample_number'])
nqdf_q1.insert(1, 'Quaternion', str('q1'), True)
nqdf_q1.insert(2, 'Q_val', df['q1'], True)

nqdf_q2=pd.DataFrame(data=df['Sample_number'])
nqdf_q2.insert(1, 'Quaternion', str('q2'), True)
nqdf_q2.insert(2, 'Q_val', df['q2'], True)

nqdf_q3=pd.DataFrame(data=df['Sample_number'])
nqdf_q3.insert(1, 'Quaternion', str('q3'), True)
nqdf_q3.insert(2, 'Q_val', df['q3'], True)

nqdf = pd.concat([nqdf_q0, nqdf_q1, nqdf_q2, nqdf_q3], ignore_index=True)
print(nqdf)

sns.lineplot(data=nqdf,x='Sample_number', y='Q_val',hue='Quaternion',lw=2)
plt.show()

In [ ]:
df=pd.read_csv(FULL_PATH_AND_CSV_FILE_02)
# print(df)

# # df = df.rename(columns={'Euler_computed [Roll, Pitch, Yaw]': 'Euler_computed'})
df = df.rename(columns={'Sample number': 'Sample_number'})
df = df.rename(columns={'Euler [Roll, Pitch, Yaw] LPMSB2': 'Euler_LPMSB2'})
df = df.rename(columns={'Quaternions [q0, q1, q2, q3] LPMS-B2': 'Quaternions_LPMSB2'})

df[['A','B','C']]=df.Euler_LPMSB2.str.split(',',expand=True)
df['A']=df['A'].apply(   lambda x: x.replace('[','')   )
df['C']=df['C'].apply(   lambda x:  x.replace(']','')   )
df['A'] = pd.to_numeric(df['A'], errors='coerce')
df['B'] = pd.to_numeric(df['B'], errors='coerce')
df['C'] = pd.to_numeric(df['C'], errors='coerce')

df[['q0','q1','q2', 'q3']]=df.Quaternions_LPMSB2.str.split(',',expand=True)
df['q0']=df['q0'].apply(   lambda x: x.replace('[','')   )
df['q3']=df['q3'].apply(   lambda x:  x.replace(']','')   )
df['q0'] = pd.to_numeric(df['q0'], errors='coerce')
df['q1'] = pd.to_numeric(df['q1'], errors='coerce')
df['q2'] = pd.to_numeric(df['q2'], errors='coerce')
df['q3'] = pd.to_numeric(df['q3'], errors='coerce')

print(df.head())# #Print head of csv


## EULER ANGLES
ndf_a=pd.DataFrame(data=df['Sample_number'])
ndf_a.insert(1, 'Euler_angle', str('alpha'), True)
ndf_a.insert(2, 'Euler_val', df['A'], True)

ndf_b=pd.DataFrame(data=df['Sample_number'])
ndf_b.insert(1, 'Euler_angle', str('beta'), True)
ndf_b.insert(2, 'Euler_val', df['B'], True)

ndf_c=pd.DataFrame(data=df['Sample_number'])
ndf_c.insert(1, 'Euler_angle', str('gamma'), True)
ndf_c.insert(2, 'Euler_val', df['C'], True)

ndf = pd.concat([ndf_a, ndf_b, ndf_c], ignore_index=True)
print(ndf)

sns.lineplot(data=ndf,x='Sample_number', y='Euler_val',hue='Euler_angle',lw=2)
plt.show()


## QUATERNIONS
nqdf_q0=pd.DataFrame(data=df['Sample_number'])
nqdf_q0.insert(1, 'Quaternion', str('q0'), True)
nqdf_q0.insert(2, 'Q_val', df['q0'], True)

nqdf_q1=pd.DataFrame(data=df['Sample_number'])
nqdf_q1.insert(1, 'Quaternion', str('q1'), True)
nqdf_q1.insert(2, 'Q_val', df['q1'], True)

nqdf_q2=pd.DataFrame(data=df['Sample_number'])
nqdf_q2.insert(1, 'Quaternion', str('q2'), True)
nqdf_q2.insert(2, 'Q_val', df['q2'], True)

nqdf_q3=pd.DataFrame(data=df['Sample_number'])
nqdf_q3.insert(1, 'Quaternion', str('q3'), True)
nqdf_q3.insert(2, 'Q_val', df['q3'], True)

nqdf = pd.concat([nqdf_q0, nqdf_q1, nqdf_q2, nqdf_q3], ignore_index=True)
print(nqdf)

sns.lineplot(data=nqdf,x='Sample_number', y='Q_val',hue='Quaternion',lw=2)
plt.show()